# Clean up the data from kibot 
* matched with polygon and pidata where possible


In [1]:
# preamble
%alias_magic  nbagg matplotlib -p nbagg
%alias_magic  inline matplotlib -p inline
%load_ext memory_profiler

from madmax.api import *
from pylab import *
mx = mx.reload()
mxtr = mxtr.reload()

# create a configuration
mx.Config.name = 'etf_technical_20200207'
mx.Config.start()
mx.Config


Created `%nbagg` as an alias for `%matplotlib nbagg`.
Created `%inline` as an alias for `%matplotlib inline`.


<IPython.core.display.Javascript object>

<h1> Config etf_technical_20200207 </h1>
<b> hash</b>: none </h4>
<b> uri </b>: /home/fj/experiments/etf_technical_20200207/none
<b> frozen </b>: False
<h3> Hyperparameters </h3>
None<h3> Code </h3>
<b> format </b>: None
<b> code </b> : None

In [2]:
# load the polygon data
@mx.operatorize( consumes_features=False, produces_features=True )
def Fix( df ):
    df = df[['adv', 'bad_adj_data', 'close', 'close_unadj', 'figi', 
             'high', 'high_unadj', 'low', 'low_unadj', 'name',
             'open', 'open_unadj', 'symbol', 'volume', 'volume_unadj']]
    df.index = df.index.tz_localize(None)
    df = df.set_index( 'symbol', append=True )
    return df, ['high', 'low', 'open', 'close']    
    

features = ['high', 'low', 'open', 'close']    
if True:
    chain = mx.Chain().set(
        data=mx.DataSource.dataframe( os.environ['HOME'] + '/data/polygon/US/price_data_correct_tz.parquet' ),  
        features=None, verbose=False,
    )
    chain += Fix().set( memoize='md' )    
    chain.append( mx.Transforms.returns( sid=1 ).leading( max_gap='5D' ).set(verbose=True, memoize='d' ), features=features )
    chain.append( mx.Transforms.returns( sid=1 ).lagging( max_gap='5D' ).set(verbose=True, memoize='d' ), features=features )
    poly_df = chain()
    
    print( 'expected LaggingReturnsComputer_332dda773 got', chain[-1].hash() )
    disp( chain )
    df2da = mx.DataFrameToDataArrayChained( float ).set(
       compactify=True, memoize='md', verbose=False,
    )   
    poly_da = df2da( data=chain[-1] )
    print( 'expected dataset_to_dataarray_433ddb86d  got ', df2da[1].hash() )
    
# display( df2da )
# # mx.MEMORY_CACHE.clean_all()

expected LaggingReturnsComputer_332dda773 got LaggingReturnsComputer_332dda773


,name,operator,run_hash,features,consumed_features,produced_features,verbose,memoize,compactify,priming
0,Fix,\n Fix ( Fix_9bb9670f6 )\n Argum...,Fix_9bb9670f6,NaN,[],"[high, low, open, close]",False,md,False,None
1,LeadingReturnsComputer,LeadingReturnsComputer ( LeadingReturnsCompute...,LeadingReturnsComputer_e10f7ccb8,"[high, low, open, close]","[high, low, open, close]","[high_leading.1_rtn, low_leading.1_rtn, open_l...",1,d,True,Priming ( ### ) \n\n Arguments\n ...
2,LaggingReturnsComputer,LaggingReturnsComputer ( LaggingReturnsCompute...,LaggingReturnsComputer_332dda773,"[high, low, open, close]","[high, low, open, close]","[high_lagging.1_rtn, low_lagging.1_rtn, open_l...",1,d,True,Priming ( ### ) \n\n Arguments\n ...


expected dataset_to_dataarray_433ddb86d  got  dataset_to_dataarray_433ddb86d


In [3]:
# load the kibot and pitrading data and compare against polygon
########################################################################

import json
from madmax.base import operatorize as mxo; reload(mxo)

# ETF metadata lists
bond_etfs = pd.read_excel( HOMEDIR+'/data/all-etfs.xlsx', sheet_name='bond-etfs' )
equity_etfs = pd.read_excel( HOMEDIR+'/data/all-etfs.xlsx', sheet_name='equity' )
vol_etfs = pd.read_excel( HOMEDIR+'/data/all-etfs.xlsx', sheet_name='vol' )

pidata_dir = os.path.join( HOMEDIR, 'data/pi/' )
pidata_files = OrderedDict( [( f.replace('.csv',''), os.path.join( pidata_dir, f ) ) for f in  os.listdir( pidata_dir ) if re.fullmatch( '.*csv', f ) ] )

kibot_eod_dir = os.path.join( HOMEDIR, 'data/kibot/etf_top200_eod_adj' )
kibot_5min_dir = os.path.join( HOMEDIR, 'data/kibot/etf_top200_5min' )
kibot_eod_files = OrderedDict( [( f.replace('\xa0','').replace('.csv',''), os.path.join( kibot_eod_dir, f ) ) for f in  os.listdir( kibot_eod_dir ) if re.fullmatch( '.*csv', f ) ] )
kibot_5min_files = OrderedDict( [( f.replace('\xa0','').replace('.csv',''), os.path.join( kibot_5min_dir, f ) ) for f in  os.listdir( kibot_5min_dir ) if re.fullmatch( '.*csv', f ) ] )

assert len( set(kibot_5min_files.keys()).difference( kibot_eod_files.keys() ) ) == len( set(kibot_eod_files.keys()).difference( kibot_5min_files.keys() ) ) == 0, 'ticker mismatch'

    
@mxo.operatorize( consumes_features=False, produces_meta=True, produces_features=False, memoize='d', uses_self=True )
def etf_data( data, ticker, output='kibot_eod', self=None ):
    '''
    process one ticker from polygon, kibot and pitrading 
    data must be the polygon data operator !!
    '''
    assert output in ['kibot_5min', 'kibot_eod', 'polygon', 'pi'], 'bad output type'
    # get metadata for the ticker
    if ticker.startswith('INCOMPLETE'):
        return None, { 'ticker':ticker }
    ticker_meta = {'ETP Name':'Not Found',  'Asset Class':'Not Found'}
    for etf_list in [bond_etfs, equity_etfs, vol_etfs]:
        if ticker in etf_list.Symbol.values:
            ticker_meta = etf_list[etf_list.Symbol==ticker].iloc[0].to_dict()
            break
    ticker_meta = dict( ticker=ticker, name=ticker_meta['ETP Name'], asset_class=ticker_meta['Asset Class'] )
    with self.timer('Reading kibot eod data'):
        kdf_eod = pd.read_csv( 
            kibot_eod_files[ticker], header=None, 
            names=['date', 'open', 'high', 'low', 'close', 'volume'] 
        ).assign( vendor='kibot_eod', **ticker_meta )    
        kdf_eod['time'] = pd.to_datetime( kdf_eod.date, format='%m/%d/%Y' ) + pd.Timedelta( 16,'H')
        kdf_eod['dollar_volume'] = kdf_eod['volume'] * kdf_eod['open']
    with self.timer( 'Reading kibot 5min data'):
        kdf_5min = pd.read_csv( 
            kibot_5min_files[ticker], header=None ,
            names=['date', 'timedelta', 'open', 'high', 'low', 'close', 'volume'] 
        ).assign( vendor='kibot_5min', **ticker_meta )
        kdf_5min['dollar_volume'] = kdf_5min['volume'] * kdf_5min['open']
        kdf_5min['time'] = pd.to_datetime( kdf_5min.date, format='%m/%d/%Y' ) + pd.to_timedelta( kdf_5min.timedelta + ':00' )    
    with self.timer( 'Reading polygon data'):
        # polygon data
        if ticker in data.index.levels[1]:
            pdf = data.loc[ (slice(None), ticker), :]        
            pdf['dollar_volume'] = pdf['volume'] * pdf['open']
            pdf['vendor'] = 'polygon_eod'
        else:
            pdf = None
    # pitrading data
    if ticker in pidata_files:        
        with self.timer( 'Reading pi data'):
            pidf = pd.read_csv( pidata_files[ticker] ).assign( vendor='pi_1min', **ticker_meta )
            pidf.columns = [c.lower() for c in pidf.columns]
            pidf = pidf.rename( columns={'time':'timedelta'} )
            pidf['time'] = (pd.to_datetime( pidf.date, format='%m/%d/%Y' ) 
                    + pd.to_timedelta( pidf.timedelta.map(lambda x: '{:02}:{:02}:00'.format( x//100, x%100 ) ) ) )
            pidf['dollar_volume'] = pidf['volume'] * pidf['open']
    else:
        pidf = None    
    if pdf is not None:
        with self.timer('matching against polygon'):
            jdf = kdf_eod.set_index( 'time' ).merge_ts( pdf.reset_index( level=1 ), suffixes=('_kibot', '_polygon'), how='inner')
            ticker_meta['kibot_eod_vs_polygon'] = { col : jdf[f'{col}_kibot'].corr( jdf[f'{col}_polygon'])  
                                                    for col in ['open','high', 'low', 'close', 'volume'] }
    if pidf is not None:
        with self.timer( 'matching against pitrading'):
            jdf = kdf_5min.set_index('time').merge_ts( pidf.set_index('time'), how='left', suffixes=('_kibot', '_pi') )
            ticker_meta['kibot_5min_vs_pi'] = { col : jdf[f'{col}_kibot'].corr( jdf[f'{col}_pi'])  for col in ['open','high', 'low', 'close', 'volume'] }
    if output == 'kibot_5min':
        return kdf_5min, ticker_meta
    elif output == 'kibot_eod':
        return kdf_eod, ticker_meta
    elif output == 'polygon':    
        return pdf, ticker_meta    
    elif output == 'pi':
        return pidf, ticker_meta
    
etf_ops = [etf_data( ticker=ticker, output='kibot_eod' ).set( verbose=True, data=chain[-1] ) for ticker in pbar( sorted( kibot_eod_files ) )]
outs = ( [op( ) for op in pbar(etf_ops) ] )


/home/fj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/fj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
etf_ops